In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [5]:
Y_train

array([6, 6, 0, ..., 7, 3, 1])

In [6]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0.01*Z)

def deriv_ReLU(Z):
    Z = Z > 0
    Z = np.maximum(Z, 0.01)
    return Z    

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    
    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max()+1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def back_prop(Z1, A1, Z2, A2, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def back_prop2(Z1, A1, Z2, A2, W2, X, Y):
    one_hot_Y = 4 * one_hot(Y)
    dA2 = 2 * (A2 - one_hot_Y)
    dZ2 = dA2
    dW2 = dZ2.dot(A1.T)
    db2 = dZ2
    dZ1 = W2.T.dot(dZ2)
    dW1 = dZ1.dot(X.T)
    db1 = dZ1
    print(db1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
#     print(alpha)
#     print(db1)
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [7]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            print("Accuracy : ", get_accuracy(get_predictions(A2), Y))
    return W1, b1, W2, b2

In [8]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 500, .1)

Iteration:  0
[2 9 3 ... 3 4 3] [6 6 0 ... 7 3 1]
Accuracy :  0.10253658536585365
Iteration:  10
[4 9 3 ... 3 7 7] [6 6 0 ... 7 3 1]
Accuracy :  0.206390243902439
Iteration:  20
[4 6 3 ... 3 0 1] [6 6 0 ... 7 3 1]
Accuracy :  0.2890243902439024
Iteration:  30
[2 6 0 ... 3 0 1] [6 6 0 ... 7 3 1]
Accuracy :  0.35885365853658535
Iteration:  40
[4 6 0 ... 3 0 1] [6 6 0 ... 7 3 1]
Accuracy :  0.42034146341463413
Iteration:  50
[4 6 0 ... 7 0 1] [6 6 0 ... 7 3 1]
Accuracy :  0.4768780487804878
Iteration:  60
[4 6 8 ... 7 0 1] [6 6 0 ... 7 3 1]
Accuracy :  0.526390243902439
Iteration:  70
[6 6 8 ... 7 0 1] [6 6 0 ... 7 3 1]
Accuracy :  0.5729512195121951
Iteration:  80
[6 6 8 ... 7 3 1] [6 6 0 ... 7 3 1]
Accuracy :  0.6097073170731707
Iteration:  90
[6 6 8 ... 7 3 1] [6 6 0 ... 7 3 1]
Accuracy :  0.6406341463414634
Iteration:  100
[6 6 8 ... 7 3 1] [6 6 0 ... 7 3 1]
Accuracy :  0.6668780487804878
Iteration:  110
[6 6 8 ... 7 3 1] [6 6 0 ... 7 3 1]
Accuracy :  0.6870487804878049
Iteration:  12